In [ ]:
from ngsolve import *
from netgen.csg import *
import netgen.meshing as meshing
import netgen.gui
%gui tk

Define geometry and $H^1$ space

In [ ]:
geo = CSGeometry()
sphere = Sphere (Pnt(0,0,0),1).bc("sphere")
plane = Plane( Pnt(0,0,0), Vec(0,-1,0))

halfsphere = sphere*plane
geo.AddSurface (sphere, halfsphere)
mesh = Mesh(geo.GenerateMesh(maxh=0.2))

order = 1
mesh.Curve(order)
Draw(mesh)

fes = H1(mesh, order=order)
print ("ndof-H1:", fes.ndof)

In [ ]:
bt = fes.FreeDofs()
print(bt)

In [ ]:
for el in fes.Elements(BBND):
    for dofs in el.dofs:
        bt.Clear(dofs)
print(bt)

\begin{align*}
\int_S\nabla_{\Gamma}u\cdot\nabla_{\Gamma}v\,ds = f(v)
\end{align*}

In [ ]:
u,v = fes.TnT()

a = BilinearForm(fes, symmetric=True)
a += SymbolicBFI ( grad(u)*grad(v), BND )
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI ( 1*v, BND )
f.Assemble()

gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(bt) * f.vec

Draw (gfu, mesh, "gfu")